# 개와 고양이 분류 (사전 학습 모델 VGG19 - 미세 조정)

이 노트북은 **미세 조정(Fine-Tuning)** 기법을 사용하여 개와 고양이 이미지를 분류합니다. 미세 조정은 사전 학습된 모델의 일부(주로 상위 레이어)를 새로운 데이터셋에 맞게 재학습시키는 고급 전이 학습 방법입니다.

**프로세스:**
1. **1단계: 특성 추출기 학습**
   - VGG19의 합성곱 기반(Convolutional Base)을 **동결(freeze)**한 상태로 두고, 그 위에 새로 추가한 분류기(Dense 레이어)만 학습시킵니다.
   - 이는 `개와고양이_사전학습2.ipynb`의 인라인 방식과 동일합니다.
2. **2단계: 미세 조정**
   - VGG19 합성곱 기반의 일부 상위 레이어의 동결을 **해제(unfreeze)**합니다.
   - 매우 낮은 학습률(learning rate)로 모델 전체를 다시 학습시킵니다. 이는 사전 학습된 가중치가 급격하게 변하는 것을 방지하고, 새로운 데이터에 맞게 세밀하게 조정하기 위함입니다.

## 1. 라이브러리 임포트

In [1]:
import os
import shutil
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, applications, optimizers
import matplotlib.pyplot as plt

## 2. 데이터 준비

### 2.1. 데이터셋 다운로드 및 정리

In [2]:
dataset_url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
zip_path = tf.keras.utils.get_file('kagglecatsanddogs.zip', origin=dataset_url, extract=True)
original_dir = pathlib.Path(zip_path).parent / 'PetImages'

# 손상된 이미지 제거
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = original_dir / folder_name
    if not folder_path.is_dir(): continue
    for fname in os.listdir(folder_path):
        fpath = folder_path / fname
        if not fpath.is_file(): continue
        try:
            with open(fpath, "rb") as fobj:
                is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        except Exception:
            is_jfif = False
        if not is_jfif:
            num_skipped += 1
            os.remove(fpath)
print(f"손상된 이미지 {num_skipped}개를 삭제했습니다.")

손상된 이미지 1578개를 삭제했습니다.


### 2.2. 훈련/검증/테스트용 서브셋 생성

In [3]:
new_base_dir = pathlib.Path("./cats_vs_dogs_small_finetune")
if not new_base_dir.exists():
    print("훈련, 검증, 테스트 서브셋을 새로 생성합니다...")
    for subset_name, start, end in [("train", 0, 1000), ("validation", 1000, 1500), ("test", 1500, 2000)]:
        for category in ("Cat", "Dog"):
            dir = new_base_dir / subset_name / category
            os.makedirs(dir, exist_ok=True)
            fnames = sorted(os.listdir(original_dir / category))[start:end]
            for fname in fnames:
                shutil.copyfile(src=original_dir / category / fname, dst=dir / fname)
else:
    print("서브셋 디렉토리가 이미 존재하여 재생성하지 않습니다.")

훈련, 검증, 테스트 서브셋을 새로 생성합니다...


### 2.3. 데이터 로더 생성

In [4]:
from tensorflow.keras.utils import image_dataset_from_directory

BATCH_SIZE = 32
IMAGE_SIZE = (180, 180)

train_ds = image_dataset_from_directory(new_base_dir / "train", image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
validation_ds = image_dataset_from_directory(new_base_dir / "validation", image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
test_ds = image_dataset_from_directory(new_base_dir / "test", image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


## 3. 모델 구성 및 1단계 학습 (분류기만 학습)

In [5]:
# VGG19 합성곱 기반 로드 및 동결
conv_base = applications.vgg19.VGG19(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)
conv_base.trainable = False

# 데이터 증강 및 전체 모델 구성
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = applications.vgg19.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4), # 1단계 학습률
              metrics=['accuracy'])

print("--- 1단계: 동결된 VGG19 기반으로 분류기 학습 ---")
history_phase1 = model.fit(train_ds, epochs=30, validation_data=validation_ds)



--- 1단계: 동결된 VGG19 기반으로 분류기 학습 ---
Epoch 1/30


63/63 [==============================] - 113s 2s/step - loss: 1.8520 - accuracy: 0.8595 - val_loss: 0.4243 - val_accuracy: 0.9640
Epoch 2/30
63/63 [==============================] - 111s 2s/step - loss: 0.7836 - accuracy: 0.9215 - val_loss: 0.4055 - val_accuracy: 0.9620
Epoch 3/30
63/63 [==============================] - 111s 2s/step - loss: 0.6026 - accuracy: 0.9350 - val_loss: 0.3207 - val_accuracy: 0.9700
Epoch 4/30
63/63 [==============================] - 112s 2s/step - loss: 0.3864 - accuracy: 0.9475 - val_loss: 0.2910 - val_accuracy: 0.9670
Epoch 5/30
63/63 [==============================] - 116s 2s/step - loss: 0.3780 - accuracy: 0.9485 - val_loss: 0.3129 - val_accuracy: 0.9640
Epoch 6/30
63/63 [==============================] - 113s 2s/step - loss: 0.2318 - accuracy: 0.9605 - val_loss: 0.2278 - val_accuracy: 0.9710
Epoch 7/30
63/63 [==============================] - 113s 2s/step - loss: 0.2946 - accuracy: 0.9555 - val_loss: 0.20

## 4. 2단계 학습 (미세 조정)

### 4.1. VGG19 상위 레이어 동결 해제

In [6]:
# conv_base를 다시 학습 가능하도록 설정
conv_base.trainable = True

# 마지막 4개 레이어(block5_conv1, block5_conv2, block5_conv3, block5_pool)만 학습
for layer in conv_base.layers[:-4]:
    layer.trainable = False

print("--- VGG19 레이어별 학습 가능 상태 ---")
for layer in conv_base.layers:
    print(f"{layer.name}: {'학습 가능' if layer.trainable else '동결'}")

--- VGG19 레이어별 학습 가능 상태 ---
input_1: 동결
block1_conv1: 동결
block1_conv2: 동결
block1_pool: 동결
block2_conv1: 동결
block2_conv2: 동결
block2_pool: 동결
block3_conv1: 동결
block3_conv2: 동결
block3_conv3: 동결
block3_conv4: 동결
block3_pool: 동결
block4_conv1: 동결
block4_conv2: 동결
block4_conv3: 동결
block4_conv4: 동결
block4_pool: 동결
block5_conv1: 동결
block5_conv2: 학습 가능
block5_conv3: 학습 가능
block5_conv4: 학습 가능
block5_pool: 학습 가능


### 4.2. 미세 조정 학습 실행

In [ ]:
# 매우 낮은 학습률로 모델을 다시 컴파일
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5), # 미세 조정을 위한 낮은 학습률
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning_model.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

print("\n--- 2단계: VGG19 상위 레이어 미세 조정 학습 ---")
history_phase2 = model.fit(
    train_ds,
    epochs=20, # 추가 에포크
    validation_data=validation_ds,
    callbacks=callbacks,
    initial_epoch=history_phase1.epoch[-1] if history_phase1.epoch else 0  # 이전 학습의 마지막 에포크에서 시작
)


--- 2단계: VGG19 상위 레이어 미세 조정 학습 ---


## 5. 학습 결과 시각화

In [8]:
acc = history_phase1.history['accuracy'] + history_phase2.history['accuracy']
val_acc = history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy']
loss = history_phase1.history['loss'] + history_phase2.history['loss']
val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.axvline(history_phase1.epoch[-1] + 1, linestyle='--', color='k', label='Fine-tuning starts')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.axvline(history_phase1.epoch[-1] + 1, linestyle='--', color='k', label='Fine-tuning starts')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

KeyError: 'accuracy'

## 6. 최종 모델 평가

In [ ]:
best_model = keras.models.load_model("fine_tuning_model.keras")

print("테스트 데이터셋으로 최종 모델을 평가합니다...")
test_loss, test_acc = best_model.evaluate(test_ds)
print(f'최종 테스트 정확도: {test_acc*100:.2f}%')